In [6]:
import jsonlines
from collections import defaultdict


In [7]:
nba_list = []
seen = set()
team_dict = defaultdict(list)
player_dict = {}

with jsonlines.open('../data_backup/player.jsonlines') as f:

    for line in f.iter():
        label = line['full_name']
        thumbnail = line['avatar_link']
        player_id = "http://dbpedia.org/resource/" + label.replace(' ', '_')
        year = line['born'][-2:]
        if year[0] == '0':
            birthYear = int('20' + line['born'][-2:])
        else:
            birthYear = int('19' + line['born'][-2:])
        salary = line['salary']
        position = line['position']

        team = line['team']
        team_id = "http://dbpedia.org/resource/" + team.replace(' ', '_')

        if label not in seen:
            player_dict[player_id] = {
                "_id": player_id,
                "label": label,
                "_type": "Person",
                "birthYear": birthYear,
                "salary": salary,
                "playsFor": [team_id],
                "position": position,
                "thumbnail": thumbnail
            }

        team_dict[team].append(player_id)
            
        seen.add(label)

    for team in team_dict:
        team_id = "http://dbpedia.org/resource/" + team.replace(' ', '_')

        nba_list.append({
                "_id": team_id,
                "_type": "Team",
                "label": team,
                "hasPlayers": team_dict[team]
            })

In [8]:
def merge_two_dicts(x, y):
    """Given two dictionaries, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [9]:
seen2 = set()
school_dict = defaultdict(list)


with jsonlines.open('../data_backup/gameinfo_modified.jsonlines') as f:

    for line in f.iter():
        label = line['full_name'][1:]
        player_id = "http://dbpedia.org/resource/" + label.replace(' ', '_')
        rating = int(line['overallRating'])
        birthCountry = line['nationality'][1:]
        school = line['school'][1:]
    
        
        if birthCountry not in seen2:
            nation_id = "http://dbpedia.org/resource/" + birthCountry.replace(' ', '_')
            nba_list.append({
                "_id": nation_id,
                "label": birthCountry,
                "_type": "Country"
            })
            seen2.add(birthCountry)

        if school in ['Spain', 'Italy', 'France', 'Australia', 'Brazil', 'Germany', 'Turkey', 'Serbia', 'Czech Republic', 'Greece']:
            school = ""
        else: 
            if school in ["USC Trojans", "Southern California"]:
                school = "USC"
            
            if school not in ['Coronado High School', 'St. Vincent\u2013St. Mary High School', 'UCLA', 'UNC', 'UNLV', 'USC', 'LSU', 'TCU', 'SMU'] :
                school += ' University'
            else:
                school = school


        if school not in ['Spain', 'Italy', 'France', 'Australia', 'Brazil', 'Germany', 'Turkey', 'Serbia', 'Czech Republic', 'Greece']:
            school_dict[school].append(player_id)

        if player_id not in player_dict.keys():
            continue
            
        if school:
            player_dict[player_id] = merge_two_dicts(player_dict[player_id], {
                    "_id": player_id,
                    "label": label,
                    "_type": "Person",
                    "hasRating": rating,
                    "birthCountry": ["http://dbpedia.org/resource/" + birthCountry.replace(' ', '_')]
,
                    "graduatedFrom": ["http://dbpedia.org/resource/" + school.replace(' ', '_')]
                })
        else:
            player_dict[player_id] = merge_two_dicts(player_dict[player_id], {
                    "_id": player_id,
                    "label": label,
                    "_type": "Person",
                    "hasRating": rating,
                    "birthCountry": ["http://dbpedia.org/resource/" + birthCountry.replace(' ', '_')]
                })
    
    for p in player_dict.keys():
        if player_id not in seen2:
            nba_list.append(player_dict[p])
            seen2.add(player_dict[p]["_id"])
    
    for s in school_dict:
        nba_list.append({
            "_id": "http://dbpedia.org/resource/" + s.replace(' ', '_'),
            "_type": "School",
            "label": s,
            "hasAlumni": school_dict[s]
        })

In [10]:
import json

with open('../data_backup/nba_full.json', 'w') as fout:
    json.dump(nba_list , fout)